### Options when creating table

In [0]:
USE CATALOG databricks_01;
USE DATABASE demo;

In [0]:
-- Create a Delta table with several advanced options
CREATE TABLE IF NOT EXISTS demo.customer_profiles (
    customer_id INT COMMENT 'Unique customer ID',
    full_name STRING,
    email STRING,
    country STRING,
    signup_date DATE DEFAULT current_date(),
    is_active BOOLEAN DEFAULT true
)
USING DELTA
PARTITIONED BY (country)
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true',
    'delta.feature.allowColumnDefaults' = 'supported',
    'description' = 'Stores profiles of customers for analytics segmentation',
    'created_by' = 'Mobilo24'
)
COMMENT 'This table holds customer profile data including personal info, region and activity flags';


In [0]:
-- Insert sample data into the customer_profiles table
INSERT INTO demo.customer_profiles (customer_id, full_name, email, country, signup_date, is_active)
VALUES
  (1001, 'Alice Novak', 'alice.novak@example.com', 'Poland', DATE '2023-01-15', true),
  (1002, 'Liam Keller', 'liam.keller@domain.org', 'Germany', DATE '2023-03-22', false),
  (1003, 'Sara Lefevre', 'sara.lefevre@example.net', 'France', DATE '2022-12-02', true),
  (1004, 'Jonas Schmidt', 'jonas.schmidt@example.de', 'Germany', DATE '2023-04-01', true),
  (1005, 'Eva Nowak', 'eva.nowak@example.pl', 'Poland', DATE '2023-05-10', false);


In [0]:
SELECT * FROM demo.customer_profiles;

In [0]:
DESCRIBE EXTENDED demo.customer_profiles;

In [0]:
DESCRIBE DETAIL demo.customer_profiles;

### Contraints

In [0]:
CREATE TABLE IF NOT EXISTS demo.customers (
  customer_id INT,
  name STRING,
  email STRING,
  age INT
)
USING DELTA
TBLPROPERTIES ('delta.enableChangeDataFeed' = 'true');

ALTER TABLE demo.customers ALTER COLUMN customer_id SET NOT NULL;
ALTER TABLE demo.customers ALTER COLUMN name SET NOT NULL;
ALTER TABLE demo.customers ALTER COLUMN age SET NOT NULL;
ALTER TABLE demo.customers ADD CONSTRAINT pk_customer PRIMARY KEY (customer_id);
ALTER TABLE demo.customers ADD CONSTRAINT check_age CHECK (age >= 18);


In [0]:
CREATE TABLE IF NOT EXISTS demo.orders (
  order_id INT,
  customer_id INT,
  amount DECIMAL(10,2),
  status STRING
)
USING DELTA;

ALTER TABLE demo.orders ALTER COLUMN order_id SET NOT NULL;
ALTER TABLE demo.orders ALTER COLUMN amount SET NOT NULL;
ALTER TABLE demo.orders ADD CONSTRAINT check_amount CHECK (amount > 0);
ALTER TABLE demo.orders ADD CONSTRAINT pk_order PRIMARY KEY (order_id);
ALTER TABLE demo.orders ADD CONSTRAINT fk_customer FOREIGN KEY (customer_id) REFERENCES demo.customers(customer_id);



In [0]:
INSERT INTO demo.customers VALUES (101, 'Jan Kowalski', 'jan.kowalski@example.com', 29);

INSERT INTO demo.orders VALUES
  (10001, 101, 199.99, 'Shipped'),
  (10002, 101, 55.50, 'Processing');


In [0]:
--  CHECK (age < 18)
--INSERT INTO demo.customers VALUES (102, 'David Avolt', 'david.avolt@example.com', 17);


In [0]:
--  NOT NULL
--INSERT INTO demo.orders VALUES (10003, 101, NULL, 'Pending');


In [0]:
-- missing foreign key customer_id
INSERT INTO demo.orders VALUES (10004, 9999, 120.00, 'Pending');



In [0]:
SELECT * FROM demo.orders;

In [0]:
DESCRIBE TABLE EXTENDED databricks_01.demo.orders;


### Create Table as Select (CTAS)

In [0]:
SELECT * FROM demo.products

In [0]:
CREATE TABLE demo.products_big 
AS SELECT 
  id,
  upper(name) AS name,
  price * 2 AS price,
  0 as amount
FROM demo.products;

In [0]:
SELECT * FROM demo.products_big;

In [0]:
DESCRIBE DETAIL demo.products_big;

In [0]:
DESCRIBE HISTORY demo.products_big;

### Cloned Table: deep vs shallow

In [0]:
SELECT * FROM demo.products;

In [0]:
-- Create a full physical copy of the original table
CREATE TABLE demo.products_deep_clone
DEEP CLONE demo.products
LOCATION 'abfss://mydata01@staccmobilo24.dfs.core.windows.net/clones/products_deep_clone';


In [0]:
-- Create a lightweight clone that shares data files with the source table
CREATE TABLE demo.products_shallow_clone
SHALLOW CLONE demo.products;


In [0]:
-- Add a new product to the original table only
INSERT INTO demo.products VALUES (8, 'Climbing shoes', 120, 5);


In [0]:
-- Query the original table (should include the new product)
SELECT * FROM demo.products ORDER BY id;

In [0]:
-- Query the deep clone (should NOT include the new product yet)
SELECT * FROM demo.products_deep_clone ORDER BY id;

In [0]:
-- Query the shallow clone (should NOT include the new product yet)
SELECT * FROM demo.products_shallow_clone ORDER BY id;

In [0]:
CREATE OR REPLACE TABLE demo.products_shallow_clone
SHALLOW CLONE demo.products;

CREATE OR REPLACE TABLE demo.products_deep_clone
DEEP CLONE demo.products
LOCATION 'abfss://mydata01@staccmobilo24.dfs.core.windows.net/clones/products_deep_clone';

In [0]:
-- Clean up both clones
DROP TABLE IF EXISTS demo.products_deep_clone;
DROP TABLE IF EXISTS demo.products_shallow_clone;


In [0]:
-- Should return all 8 products
SELECT * FROM demo.products ORDER BY id;


### Views

In [0]:
USE CATALOG databricks_01;
USE SCHEMA demo;

saved in the schema (database)

In [0]:
CREATE OR REPLACE VIEW cheap_products_50 AS
SELECT * FROM products WHERE price < 50;


In [0]:
SELECT * FROM cheap_products_50;

lives so long as the session

In [0]:
CREATE OR REPLACE TEMP VIEW cheap_products_40 AS
SELECT * FROM products WHERE price < 40;


In [0]:
SELECT * FROM cheap_products_40;

lives so long as the cluster


In [0]:
CREATE OR REPLACE GLOBAL TEMP VIEW cheap_products_30 AS
SELECT * FROM products WHERE price < 30;

In [0]:
SELECT * FROM global_temp.cheap_products_30;

### Metadata for VIEW

In [0]:
SHOW TABLES

In [0]:
SHOW VIEWS;

In [0]:
SHOW VIEWS IN global_temp;

--> compare in another notebook

In [0]:
SELECT * FROM information_schema.views WHERE table_schema = CURRENT_DATABASE();